In [31]:
import pandas as pd
import utm
from utils.preprocessing import preprocess
from utils.from_latlon import from_latlon
import math


In [149]:
data_type = ['Simulated BTFS', 'BTFS']
freq = '3min'
routine = 'training'
dimensions = ['xOffset', 'yOffset']
sm = 150 # maximum signal difference between min and max, used for calibration

In [12]:
def preprocess_sim_data(sim_data, data_type, freq, tower_locs, routine):
    # Get data
    sim_dat_filt = sim_data[sim_data['Data_type'].isin(data_type)]
    
    sim_dat_filt, predictors = preprocess(sim_dat_filt, freq, routine)
     
    # Calculate easting and northing from lat long
    sim_dat_filt['easting'], sim_dat_filt['northing'], sim_dat_filt['zone_num'], sim_dat_filt['zone_letter'] = from_latlon(sim_dat_filt['POINT_Y'].values, sim_dat_filt['POINT_X'].values)

    # Create a dictionary of the coordinates of the towers
    offset_dict = tower_locs.set_index('TowerID').to_dict()
    point_x = offset_dict['POINT_X']
    point_y = offset_dict['POINT_Y']

    # Standardise the coordinates so that the tower location == 0 on both the x and y axes.
    sim_dat_filt['xOffset'] = sim_dat_filt['easting'] - sim_dat_filt['TowerID'].map(point_x).fillna(0)
    sim_dat_filt['yOffset'] = sim_dat_filt['northing'] - sim_dat_filt['TowerID'].map(point_y).fillna(0)
    
    return sim_dat_filt, predictors

In [4]:
test_data = 'Example_data\Output\Train_test_data\Testing_Tag_GPS_locations.xlsx'

# Get testing data
test_data = pd.read_excel(test_data)
test_data['DateAndTime'] = pd.to_datetime(test_data['DateAndTime'])

# Get tower locations
radio_tower_xy_path = 'Example_data\Input\Radio_tower_locations\RTEastNorth.xlsx'
tower_locs = pd.read_excel(radio_tower_xy_path)

In [13]:
# Create test dataset as per train_model_h2o method
test_data_preproc, predictors_test = preprocess_sim_data(test_data, data_type, freq, tower_locs, routine)


In [14]:
test_data_preproc

,DateTime,TowerID,TagID,Data_type,POINT_X,POINT_Y,Point_ID,Tag_type,Interval_seconds,ant1_count,...,ant3_std,ant4_std,mean_std,total_count,easting,northing,zone_num,zone_letter,xOffset,yOffset
0,2021-02-02 07:51:00,RT01,60,BTFS,146.256427,-21.919968,299,Nanotag,13,0.0,...,0.000000,0.000000,11.313708,3.0,423203.210195,7.575845e+06,55,K,-135.520989,-73.105345
1,2021-02-02 07:51:00,RT04,60,BTFS,146.256427,-21.919968,299,Nanotag,13,1.0,...,0.000000,0.000000,0.000000,1.0,423203.210195,7.575845e+06,55,K,257.092334,171.458690
2,2021-02-02 07:54:00,RT01,60,BTFS,146.256427,-21.919968,299,Nanotag,13,0.0,...,0.000000,8.386497,5.961015,5.0,423203.210195,7.575845e+06,55,K,-135.520989,-73.105345
3,2021-02-02 07:54:00,RT04,60,BTFS,146.256427,-21.919968,299,Nanotag,13,3.0,...,11.056672,5.773503,12.546276,10.0,423203.210195,7.575845e+06,55,K,257.092334,171.458690
4,2021-02-04 18:36:00,RT12,52,BTFS,146.295603,-22.016401,303,Nanotag,13,2.0,...,1.414214,0.000000,2.474874,4.0,427298.584493,7.565190e+06,55,K,53.909953,-458.008627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,2023-05-23 10:42:00,RT18,192,BTFS,146.237257,-21.938164,551,Nanotag,3,15.0,...,4.882427,3.254751,4.621087,58.0,421233.248221,7.573822e+06,55,K,103.510404,-165.520759
612,2023-05-23 10:45:00,RT18,192,BTFS,146.237257,-21.938164,551,Nanotag,3,15.0,...,16.183246,1.027105,12.738204,56.0,421233.248221,7.573822e+06,55,K,103.510404,-165.520759
613,2023-05-23 10:48:00,RT18,192,BTFS,146.237257,-21.938164,551,Nanotag,3,14.0,...,10.454769,7.749878,8.238318,54.0,421233.248221,7.573822e+06,55,K,103.510404,-165.520759
614,2023-05-23 10:51:00,RT18,192,BTFS,146.237257,-21.938164,551,Nanotag,3,11.0,...,13.776792,6.142746,11.460246,44.0,421233.248221,7.573822e+06,55,K,103.510404,-165.520759


In [15]:
test_data_preproc.ant1_mean


0        0.000000
1      117.000000
2        0.000000
3      140.000000
4       68.500000
          ...    
611    102.200000
612    101.533333
613     98.285714
614     92.454545
615     73.166667
Name: ant1_mean, Length: 616, dtype: float64

In [163]:
def calculate_delta_g(s1, s2, sm):
    delta_g = (s1-s2)/sm
    return delta_g

def calculate_offset_angle(delta_g):
    bearing = math.acos(delta_g) * 90/math.pi
    return bearing

In [122]:
s1 = 160
s2 = 160
sm = 180

delta_g = (s1-s2)/sm
print(f'delta_g = {delta_g}')
print(f'math.acos(delta_g) = {math.acos(delta_g)}')

result = (math.pi)/90 * math.acos(delta_g)
print(f'result = {result}')

result = math.acos(delta_g) * 90/math.pi
print(f'result = {result}')



delta_g = 0.0
math.acos(delta_g) = 1.5707963267948966
result = 0.05483113556160755
result = 45.0


In [ ]:
## Calculate total paired antenna values to find maximum sum, only if both values do not equal zero.

# Add 'ant_1_2_sum' column
test_data_preproc['ant_1_2_sum'] = test_data_preproc.apply(lambda row: row['ant1_mean'] + row['ant2_mean'] if row['ant1_mean'] != 0 and row['ant2_mean'] != 0 else 'na', axis=1)

# Add 'ant_2_3_sum' column
test_data_preproc['ant_2_3_sum'] = test_data_preproc.apply(lambda row: row['ant2_mean'] + row['ant3_mean'] if row['ant2_mean'] != 0 and row['ant3_mean'] != 0 else 'na', axis=1)

# Add 'ant_3_4_sum' column
test_data_preproc['ant_3_4_sum'] = test_data_preproc.apply(lambda row: row['ant3_mean'] + row['ant4_mean'] if row['ant3_mean'] != 0 and row['ant4_mean'] != 0 else 'na', axis=1)

# Add 'ant_4_1_sum' column
test_data_preproc['ant_4_1_sum'] = test_data_preproc.apply(lambda row: row['ant4_mean'] + row['ant1_mean'] if row['ant4_mean'] != 0 and row['ant1_mean'] != 0 else 'na', axis=1)



In [180]:
#### Calculate the bearings from the tower

# List of antenna names
antenna_names = ['ant_1_2_sum', 'ant_2_3_sum', 'ant_3_4_sum', 'ant_4_1_sum']

# Create a new column 'bearing' in the DataFrame
test_data_preproc['bearing'] = None

# Iterate through the DataFrame
for index, row in test_data_preproc.iterrows():
    # Extract values from the new columns and handle 'na' values
    values = [float(row['ant_1_2_sum']) if row['ant_1_2_sum'] != 'na' else float('-inf'),
              float(row['ant_2_3_sum']) if row['ant_2_3_sum'] != 'na' else float('-inf'),
              float(row['ant_3_4_sum']) if row['ant_3_4_sum'] != 'na' else float('-inf'),
              float(row['ant_4_1_sum']) if row['ant_4_1_sum'] != 'na' else float('-inf')]
    
    # Check if all values are 'na'; if yes, skip the row
    if all(val == float('-inf') for val in values):
        continue
    
    # Find the column name with the highest value
    max_column = max(zip(values, antenna_names))[1]
        
    if max_column == "ant_1_2_sum":
        if row.ant1_mean >= row.ant2_mean:
            delta_g = calculate_delta_g(row.ant1_mean, row.ant2_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)
            bearing = offset_angle
            
            # print(f'delta_g = {delta_g}')
            # print(f'offset_angle from ant 1 = {offset_angle}')
            # print(f'bearing ant 1 = {bearing}')

        else:
            delta_g = calculate_delta_g(row.ant2_mean, row.ant1_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)          
            bearing = 90 - offset_angle

    elif max_column == "ant_2_3_sum":
        if row.ant2_mean >= row.ant3_mean:
            delta_g = calculate_delta_g(row.ant2_mean, row.ant3_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)
            bearing = 90 + offset_angle

        else:
            delta_g = calculate_delta_g(row.ant3_mean, row.ant2_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)          
            bearing = 180 - offset_angle

    elif max_column == "ant_3_4_sum":
        if row.ant3_mean >= row.ant4_mean:
            delta_g = calculate_delta_g(row.ant3_mean, row.ant4_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)
            bearing = 180 + offset_angle

        else:
            delta_g = calculate_delta_g(row.ant4_mean, row.ant3_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)          
            bearing = 270 - offset_angle

    elif max_column == "ant_4_1_sum":
        if row.ant4_mean >= row.ant1_mean:
            delta_g = calculate_delta_g(row.ant4_mean, row.ant1_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)
            bearing = 270 + offset_angle

        else:
            delta_g = calculate_delta_g(row.ant1_mean, row.ant4_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)          
            bearing = 360 - offset_angle
    
    
    # Assign the calculated bearing to the 'bearing' column
    test_data_preproc.at[index, 'bearing'] = bearing

In [181]:
test_data_preproc.bearing

0            None
1            None
2            None
3      317.741648
4            None
          ...    
611     136.36288
612     137.31218
613    135.559351
614    136.002881
615    131.602038
Name: bearing, Length: 616, dtype: object

In [ ]:
# Triangulation

In [172]:
test_data_preproc

,DateTime,TowerID,TagID,Data_type,POINT_X,POINT_Y,Point_ID,Tag_type,Interval_seconds,ant1_count,...,ant_second_strongest,ant_max,ant_max_name,ant_2max,ant_2max_name,ant_1_2_sum,ant_2_3_sum,ant_3_4_sum,ant_4_1_sum,max_column
0,2021-02-02 07:51:00,RT01,60,BTFS,146.256427,-21.919968,299,Nanotag,13,0.0,...,119.0,121.000000,ant2_mean,119.0,ant4_mean,na,na,na,na,ant4_mean
1,2021-02-02 07:51:00,RT04,60,BTFS,146.256427,-21.919968,299,Nanotag,13,1.0,...,<NA>,117.000000,ant1_mean,<NA>,<NA>,na,na,na,na,ant4_mean
2,2021-02-02 07:54:00,RT01,60,BTFS,146.256427,-21.919968,299,Nanotag,13,0.0,...,120.5,128.666667,ant4_mean,120.5,ant2_mean,na,na,na,na,ant4_mean
3,2021-02-02 07:54:00,RT04,60,BTFS,146.256427,-21.919968,299,Nanotag,13,3.0,...,130.25,140.000000,ant1_mean,130.25,ant3_mean,na,na,255.916667,265.666667,NaN
4,2021-02-04 18:36:00,RT12,52,BTFS,146.295603,-22.016401,303,Nanotag,13,2.0,...,68.5,89.000000,ant3_mean,68.5,ant1_mean,na,na,na,na,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,2023-05-23 10:42:00,RT18,192,BTFS,146.237257,-21.938164,551,Nanotag,3,15.0,...,109.0,116.133333,ant3_mean,109.0,ant2_mean,211.2,225.133333,216.990476,203.057143,NaN
612,2023-05-23 10:45:00,RT18,192,BTFS,146.237257,-21.938164,551,Nanotag,3,15.0,...,106.214286,118.307692,ant3_mean,106.214286,ant2_mean,207.747619,224.521978,222.450549,205.67619,NaN
613,2023-05-23 10:48:00,RT18,192,BTFS,146.237257,-21.938164,551,Nanotag,3,14.0,...,112.0,114.928571,ant3_mean,112.0,ant2_mean,210.285714,226.928571,210.595238,193.952381,NaN
614,2023-05-23 10:51:00,RT18,192,BTFS,146.237257,-21.938164,551,Nanotag,3,11.0,...,97.75,103.000000,ant3_mean,97.75,ant2_mean,190.204545,200.75,193.8,183.254545,NaN
